In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
!pip install --upgrade tensorflow-io

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3045392042489765454
xla_global_id: -1
]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
    Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0
  Attempting uninstall: tensorflow-io
    Found existing installation: tensorflow-io 0.31.0
    Uninstalling tensorflow-io-0.31.0:
      Successfully uninstalled tensorflow-io-0.31.0


In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [3]:
df = pd.read_csv("/kaggle/input/merged-file/merged_T_rain.csv",encoding="utf-8")

In [4]:
df.head(n=2)

,Unnamed: 0.1,Unnamed: 0,0,1,2,3,4,5,6,7,...,72,73,74,75,76,77,78,79,80,81
0,0,0,2541497,56,26325,22294,9018,25604,8390,27941,...,1.713364,0.000000,0.115692,1.156922,0.269948,0.0,0.000000,0.0,0,0
1,1,1,2541785,50,26325,22294,9018,25604,12831,27941,...,0.571121,0.571121,0.115692,1.002666,0.269948,0.0,37.384575,0.0,1,0


In [5]:
df.drop(["Unnamed: 0","Unnamed: 0.1","24","25","27","28","29","0","1","80"],axis=1,inplace=True)
# df.drop(["Unnamed: 0.2","Unnamed: 0.1","Unnamed: 0","0","1","80"],axis=1,inplace=True)

In [6]:
df.head()

,2,3,4,5,6,7,8,9,10,11,...,71,72,73,74,75,76,77,78,79,81
0,26325,22294,9018,25604,8390,27941,21218,869,19343,12213,...,0.000000,1.713364,0.000000,0.115692,1.156922,0.269948,0.0,0.000000,0.0,0
1,26325,22294,9018,25604,12831,27941,21218,21533,19343,27554,...,0.571121,0.571121,0.571121,0.115692,1.002666,0.269948,0.0,37.384575,0.0,0
2,20095,563,31686,19475,590,27941,18800,23218,21574,31057,...,0.000000,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.000000,0.0,0
3,25976,22294,32134,21545,26733,27941,21218,21533,19343,31863,...,0.000000,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.000000,0.0,0
4,23611,22294,29534,21545,25853,27941,19606,869,22970,27554,...,0.000000,0.571121,0.571121,0.000000,1.156922,0.269948,0.0,0.000000,0.0,0


In [7]:
print(df.isnull().sum().any()>=1)

True


In [8]:
len(df.columns)

74

In [9]:
df.shape

(3485852, 74)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1:], test_size=0.1)

In [11]:
# Combine X_train and y_train into a single train dataframe
train_df = pd.concat([X_train, y_train], axis=1)

# # Combine X_test and y_test into a single test dataframe
test_df = pd.concat([X_test, y_test], axis=1)


In [12]:
del X_test,X_train,y_train,y_test,df

In [13]:
import gc
gc.collect()

0

In [14]:
# train_df['81'].any()==1

In [15]:
import tensorflow_decision_forests as tfdf
import pandas as pd
import tensorflow as tf

In [16]:
label_train = train_df['81']
label_test = test_df['81']
train_df.drop(['81'],axis=1,inplace=True)
test_df.drop(['81'],axis=1,inplace=True)
train_data = tf.data.Dataset.from_tensor_slices((dict(train_df), label_train)).batch(128)
test_data = tf.data.Dataset.from_tensor_slices((dict(test_df), label_test)).batch(128)

In [17]:
del train_df,test_df

In [18]:
import gc
gc.collect()

0

In [19]:
# train_data

In [20]:
# # Categorical features (columns 2 to 32)
# categorical_columns = [str(i) for i in range(2, 33) if str(i) in df.columns]
# categorical_feature_columns = [
#     tf.feature_column.categorical_column_with_identity(column,num_buckets=10)
#     for column in categorical_columns
# ]

# # Binary features (columns 33 to 41)
# binary_columns = [str(i) for i in range(33, 42) if str(i) in df.columns]
# binary_feature_columns = [
#     tf.feature_column.numeric_column(column)
#     for column in binary_columns
# ]

# # Numerical features (columns 42 to 79)
# numeric_columns = [str(i) for i in range(42, 80) if str(i) in df.columns]
# numeric_feature_columns = [
#     tf.feature_column.numeric_column(column)
#     for column in numeric_columns
# ]

In [21]:
# num_features = 73
# num_epochs = 100

# train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(128)
# test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(128)

# # Define the logistic regression model
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(num_features,))
# ])

# # Compile the model
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# # Train the model
# model.fit(train_data, epochs=num_epochs)

# # Evaluate the model
# test_loss, test_accuracy = model.evaluate(test_data)

In [22]:
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.CLASSIFICATION)

# model = tfdf.keras.GradientBoostedTreesModel(task=tfdf.keras.Task.CLASSIFICATION)



Use /tmp/tmpjdrgn7gh as temporary training directory


In [23]:
model.compile(metrics=["accuracy"])
model.fit(train_data)

Reading training dataset...
Training dataset read in 0:01:37.656506. Found 3137266 examples.
Training model...


[INFO 23-06-13 19:44:53.5984 UTC kernel.cc:1242] Loading model from path /tmp/tmpjdrgn7gh/model/ with prefix a2ad3473b13747a9
[INFO 23-06-13 19:45:01.5221 UTC decision_forest.cc:660] Model loaded with 300 root(s), 4718128 node(s), and 72 input feature(s).
[INFO 23-06-13 19:45:01.5221 UTC abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-06-13 19:45:01.5222 UTC kernel.cc:1074] Use fast generic engine


Model trained in 0:30:20.182657
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [24]:
# results = model.evaluate(test_data)
# print(f"Accuracy: {results}")

In [25]:
# predictions = model.predict(test_data)

In [26]:
# predictions

In [27]:
# model = tfdf.keras.RandomForestModel()
# model.fit(train_ds)

In [28]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (73):
	10
	11
	12
	13
	14
	15
	16
	17
	18
	19
	2
	20
	21
	22
	23
	26
	3
	30
	31
	32
	33
	34
	35
	36
	37
	38
	39
	4
	40
	41
	42
	43
	44
	45
	46
	47
	48
	49
	5
	50
	51
	52
	53
	54
	55
	56
	57
	58
	59
	6
	60
	61
	62
	63
	64
	65
	66
	67
	68
	69
	7
	70
	71
	72
	73
	74
	75
	76
	77
	78
	79
	8
	9

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "59"  0.117566 ################
    2.  "2"  0.102175 ###########
    3. "51"  0.101680 ###########
    4. "58"  0.095405 #########
    5. "20"  0.089127 #######
    6. "31"  0.088160 ######
    7. "68"  0.086725 ######
    8.  "6"  0.086530 ######
    9.  "4"  0.085120 ######
   10. "

In [29]:
# model.evaluate(test_ds)

In [30]:
model.save('path/to/model')

In [31]:
to_do_sheet = pd.read_csv('/kaggle/input/recsys/sharechat_recsys2023_data/T_est/000000000000.csv',encoding='utf-8')

In [32]:
row_numbers = to_do_sheet['0']
rows_with_nan = to_do_sheet.isnull().any(axis=1)
indexes_with_nan = to_do_sheet[rows_with_nan].index
print(indexes_with_nan)

Int64Index([     0,      2,      5,      6,     14,     15,     16,     17,
                27,     28,
            ...
            160937, 160940, 160941, 160944, 160953, 160958, 160959, 160962,
            160964, 160967],
           dtype='int64', length=51128)


In [33]:
to_do_sheet.fillna(to_do_sheet.median(),inplace=True)

In [34]:
to_do_sheet.drop(["Unnamed: 0","24","25","27","28","29"],axis=1,inplace=True)

In [35]:
to_do_sheet.drop(["0","1"],axis=1,inplace=True)

In [36]:
to_do_sheet.head()

,2,3,4,5,6,7,8,9,10,11,...,70,71,72,73,74,75,76,77,78,79
0,26325,7152,21563,19475,31440,27941,21621,14659,21574,31460,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,20095,563,22861,19475,21280,27941,19203,14659,21574,27957,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,890,22294,18294,21545,20210,27941,18800,9638,19343,30375,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,20095,563,22861,25604,21280,27941,18800,14659,22970,31863,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,27426,22294,11338,19475,23855,27941,21218,9638,21574,30778,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [37]:
len(to_do_sheet.columns)

73

In [38]:
# get_D_tds = xgb.DMatrix(to_do_sheet, label=None)

In [39]:
# to_do_sheet['81'] = 1

In [40]:
to_do_sheet.head()

,2,3,4,5,6,7,8,9,10,11,...,70,71,72,73,74,75,76,77,78,79
0,26325,7152,21563,19475,31440,27941,21621,14659,21574,31460,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,20095,563,22861,19475,21280,27941,19203,14659,21574,27957,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,890,22294,18294,21545,20210,27941,18800,9638,19343,30375,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,20095,563,22861,25604,21280,27941,18800,14659,22970,31863,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,27426,22294,11338,19475,23855,27941,21218,9638,21574,30778,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [41]:
# to_do_sheet.drop(['81'],axis=1,inplace=True)

get_D_s = tfdf.keras.pd_dataframe_to_tf_dataset(to_do_sheet)

# get_D_s = tf.data.Dataset.from_tensor_slices((dict(to_do_sheet),to_do_sheet['81'])).batch(128)

In [42]:
# batch_size = 128

# test_dataset = get_D_s.batch(batch_size).prefetch(1)
# get_D_s

In [43]:
# test_data

In [44]:
model.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (73):
	10
	11
	12
	13
	14
	15
	16
	17
	18
	19
	2
	20
	21
	22
	23
	26
	3
	30
	31
	32
	33
	34
	35
	36
	37
	38
	39
	4
	40
	41
	42
	43
	44
	45
	46
	47
	48
	49
	5
	50
	51
	52
	53
	54
	55
	56
	57
	58
	59
	6
	60
	61
	62
	63
	64
	65
	66
	67
	68
	69
	7
	70
	71
	72
	73
	74
	75
	76
	77
	78
	79
	8
	9

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1. "59"  0.117566 ################
    2.  "2"  0.102175 ###########
    3. "51"  0.101680 ###########
    4. "58"  0.095405 #########
    5. "20"  0.089127 #######
    6. "31"  0.088160 ######
    7. "68"  0.086725 ######
    8.  "6"  0.086530 ######
    9.  "4"  0.085120 ######
   10. "

In [45]:
submitting_list = model.predict(get_D_s)

161/161 [==============================] - 11s 68ms/step


In [46]:
submitting_list.max(),submitting_list.min(),len(submitting_list)

(0.88666594, 0.0, 160973)

In [47]:
submitting_list.resize(len(submitting_list))

In [48]:
submitting_list

array([0.06666667, 0.09666664, 0.02      , ..., 0.05      , 0.02      ,
       0.00333333], dtype=float32)

In [49]:
final_file = pd.DataFrame({'row_id':row_numbers,'is_clicked':submitting_list,'is_installed':submitting_list})

In [50]:
final_file.head()

,row_id,is_clicked,is_installed
0,64505,0.066667,0.066667
1,64506,0.096667,0.096667
2,64507,0.020000,0.020000
3,64508,0.100000,0.100000
4,64509,0.000000,0.000000


In [51]:
final_file.to_csv('new_file_tf-rt.tsv', sep='\t', index=False)